### Sample notebook for prediction against image files  

#### Import libraries  

In [ ]:
import os,glob
from PIL import Image
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

#### Parameters  

In [ ]:
w=56
work_dir="data/dogs_cats"
image_dir=work_dir+"/original/unknown"
pred_dir=work_dir+"/predicted"
results_base='dogs_cats-results'
model_file="dogs_cats-model-w{0}.h5".format(w)
classes=["dogs","cats"]
num_classes=len(classes)

#### Read model file  

In [ ]:
model=load_model(model_file)

#### Prediction against an image file one by one  

In [ ]:
# Open file for saving results  
results_file='{0}-w{1}.csv'.format(results_base,w)
res=open(results_file,'w')
res.write("file,{0},{1}\n".format(classes[0],classes[1]))

# Remove files in output directory
files=glob.glob(pred_dir+"/*.*")
for f in files:
    os.remove(f)

# Do prediction 
files=glob.glob(image_dir+"/*.*")
for f in files:
    # Read image file
    img=Image.open(f)
    img=img.convert("RGB")
    img=img.resize((w,w))
    # Reshape into 4-d ndarray
    X=np.array(img).reshape(1,w,w,3).astype("float16")
    X_pred=X/255
    # Classification prediction
    pred=model.predict(X_pred)
    # Output image into pred_dir
    pred_ans=pred.argmax()
    pred_cls=classes[pred_ans]
    tag="pred_as_{0}-w{1}".format(pred_cls,w)
    base=os.path.basename(f)
    img_file="{0}/{1}.{2}.jpg".format(pred_dir,os.path.splitext(base)[0],tag)
    img.save(img_file)
    print(img_file,pred)
    res.write("{0},{1},{2}\n".format(f,pred[0][0],pred[0][1]))

# Close results file
res.close()

print("OK")